In [25]:
import pandas as pd
import twint
import numpy as np
import nest_asyncio
nest_asyncio.apply()
import datetime
import ast
from newspaper import Article
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
import re
import swifter
from urllib.parse import urlparse
from tqdm.auto import tqdm
tqdm.pandas()
import requests

In [27]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [2]:
df = pd.read_csv("KathmanduPost.csv", parse_dates=['created_at'])

In [3]:
df

,id,created_at,urls,article
0,1376943554061348864,2021-03-30 22:50:00+05:45,http://tkpo.st/31u5qcO,National A Qatari woman aims to climb Everest ...
1,1376880639417323524,2021-03-30 18:40:00+05:45,https://tkpo.st/3sBwym6,World Myanmar protesters launch 'garbage strik...
2,1376861721130790912,2021-03-30 17:24:49+05:45,https://tkpo.st/3fnEiEw,"Health Nepal’s Covid-19 toll reaches 3,030 wit..."
3,1376807388439699456,2021-03-30 13:48:55+05:45,https://tkpo.st/31xfdPe,National Dahal files petition at Supreme Court...
4,1376800108574957569,2021-03-30 13:20:00+05:45,https://tkpo.st/39qOokl,Sports Nepal edge Bangladesh to claim Three Na...
...,...,...,...,...
29475,947657136892346368,2018-01-01 08:19:02+05:45,http://kathmandupost.ekantipur.com/news/2018-0...,National Deuba defiant despite debaclePrime Mi...
29476,947654359294382080,2018-01-01 08:08:00+05:45,http://kathmandupost.ekantipur.com/news/2018-0...,National Poll costs faked as spending goes thr...
29477,947652345613246464,2018-01-01 08:00:00+05:45,http://kathmandupost.ekantipur.com/news/2017-1...,"Special Supplement A paper trailIn 2013, Bhint..."
29478,947637246810775552,2018-01-01 07:00:00+05:45,http://kathmandupost.ekantipur.com/news/2017-1...,Special Supplement Rewriting myths for the mod...


In [4]:
# # todo change this to func
# def filter_url(url):
#     return ((urlparse(url).netloc)=="www.nepalitimes.com")

In [5]:
# df=df[df.urls.apply(filter_url)]

In [6]:
# def clean_text(raw):
#     text = BeautifulSoup(raw, 'lxml').text
#     word_tokens = word_tokenize(text.lower().rstrip())
#     filtered_sentence = [w for w in word_tokens if not w in stop_words]
#     return ' '.join(filtered_sentence)

In [4]:
def article_ktmpost(url):
    try:
        req = requests.get(url, headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        result = soup.find("div", {"class": "col-sm-8"})
        return result.text
    except Exception as e:
        print (str(e))
        return 404

In [5]:
def primary_url(urls):
    if len(urls)==0:
        return 404
    return urls[0]

In [18]:
def get_new_articles(newspaper_username, since, until):
    c = twint.Config()
    c.Username = newspaper_username
    c.Pandas = True
    c.Hide_output= True
    c.Since = since
    c.Until = until
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    try: 
        df = df[['id', 'created_at', "urls"]]
        df['urls'] = df.urls.apply(primary_url)
        df = df[df.urls!=404]
#         df = df[df.urls.apply(filter_url)]
        df['created_at'] = (pd.to_datetime(df['created_at'], unit='ms')
                         .dt.tz_localize('utc')
                         .dt.tz_convert('Asia/Kathmandu'))   
        return df
    except:
        return pd.DataFrame(columns=['id', 'created_at', "urls"])

In [19]:
data_level1 = [
    'arabl land',
    'avail data',
    'bureau statist',
    'busi survey',
    'cens publi',
    'cens pop',
    'children employ',
    'civil registr',
    'collect method',
    'commerci export',
    'complet rate',
    'consum electr',
    'consum energ',
    'data access',
    'data collect',
    'data compil',
    'data entri',
    'data manag',
    'data releas',
    'data standard',
    'data user',
    'demograph data',
    'densit popul',
    'develop data',
    'difusion dat',
    'direct statist',
    'disaggreg data',
    'electr access',
    'electr consumpt',
    'energi consumpt',
    'establish survey',
    'exchang rate',
    'extern debt',
    'fertil rate',
    'food import',
    'food product',
    'gender gap',
    'govern debt',
    'govern statist',
    'gross domest',
    'gross nation',
    'health expenditur',
    'health survey',
    'import marchandis',
    'improv data',
    'improv statist',
    'indic measur',
    'indic preci',
    'inflat rate',
    'institut statist',
    'interest payment',
    'intern tourism',
    'irrig land',
    'land use',
    'life expect',
    'livestock product',
    'merchandis export',
    'merchandis trade',
    'model statist',
    'mortal rate',
    'multilater debt',
    'nation account',
    'nation statist',
    'nation survey',
    'national brut',
    'national statist',
    'open data',
    'part revenus',
    'pay gap',
    'popul census',
    'popul growth',
    'popul rate',
    'price index',
    'produccion aliment',
    'purchas power',
    'qualiti data',
    'receit fiscal',
    'releas data',
    'revenu fiscal',
    'rural popul',
    'servic export',
    'statist agenc',
    'statist author',
    'statist avail',
    'statist committe',
    'statist data',
    'statist depart',
    'statist national',
    'statist offic',
    'statist servic',
    'statist studi',
    'survey catalogu',
    'tax payment',
    'tax revenu',
    'trade balanc',
    'unemploy rate',
    'use data',
    'water suppli',
    'youth unemploy',
    ]

data_level2 = [
    'accur',
    'adequ',
    'ambigu',
    'apropi',
    'bancal',
    'bias',
    'confiabl',
    'correct',
    'deceit',
    'deceiv',
    'decept',
    'defectu',
    'delud',
    'engan',
    'equivoc',
    'erreur',
    'erro',
    'erron',
    'errone',
    'error',
    'exact',
    'exat',
    'fake',
    'fallaci',
    'faux',
    'fiabl',
    'generaliz',
    'illus',
    'imparcial',
    'impartial',
    'imprecis',
    'improp',
    'inaccur',
    'incorrect',
    'inexact',
    'invalid',
    'limit',
    'manipul',
    'mislead',
    'mistaken',
    'parcial',
    'prec',
    'precis',
    'proper',
    'reliabl',
    'rigor',
    'rigour',
    'scientif',
    'spurious',
    'tromp',
    'trompeur',
    'unbias',
    'unreli',
    'unscientif',
    'unsound',
    'vag',
    'vagu',
    'val',
    'valid',
    ]

data_level3 = [
    'data manipul',
    'lead question',
    'manipul dat',
    'report bias',
    'sampl select',
    'sampl size',
    ]

data_level_indicator = [
    ' cpi ',
    ' fdi ',
    ' gdp ',
    ' gnp ',
    ' hdi ',
    ' wdi ',
    ]

filter_list = [' data ', ' record ', ' research ', ' statistics ',
               ' study ']

In [20]:
def level1_count(article):
    if article:
        article = str(article)
        keyword_list = []
        for word in data_level1:
            search_ = r"\b" + word.split()[0] + r"[a-zA-Z]*\s\b" \
                + word.split()[1] + r"[a-zA-Z]*"
            if re.search(search_, article):
                keyword_list.append(word)
        for word in data_level_indicator:
            if word in article:
                keyword_list.append(word)
        return keyword_list
    return []


def level2_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level2:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,  article):
                keyword_list.append(word)
        return keyword_list
    return []


def level3_count(article, valid):
    if valid == 1:
        article = str(article)
        keyword_list = []
        for word in data_level3:
            search_ = r"\b" + word + r"[a-zA-Z]*"
            if re.search(search_,   article):
                keyword_list.append(word)
        return keyword_list
    return []


def level_2_3_filter(article):
    article = str(article)
    for word in filter_list:
        if word in article:
            return 1
    return 0


def level_len(count_list):
    if type(count_list) == str:
        return (1 if len(count_list) > 2 else 0)
    return (1 if len(count_list) > 0 else 0)

In [21]:
df_ = get_new_articles(newspaper_username='kathmandupost', 
                       since=df.created_at.max().strftime("%Y-%m-%d %H:%M:%S"),
                       until=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


<ipython-input-18-bd3985bb40f2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['urls'] = df.urls.apply(primary_url)


In [22]:
df_

,id,created_at,urls
0,1393698959324028930,2021-05-16 04:30:00+05:45,http://tkpo.st/3tMY9jZ
1,1393668760322539521,2021-05-16 02:30:00+05:45,http://tkpo.st/2RbYAax
2,1393631011800449024,2021-05-16 00:00:00+05:45,http://tkpo.st/2RZnTN2
3,1393615912071307267,2021-05-15 23:00:00+05:45,http://tkpo.st/3tNunvz
4,1393600812832657409,2021-05-15 22:00:00+05:45,http://tkpo.st/3eSSa9j
...,...,...,...
1249,1377099581977030659,2021-03-31 09:10:00+05:45,https://tkpo.st/31yTQgs
1250,1377093297546657792,2021-03-31 08:45:01+05:45,https://tkpo.st/3udXpVT
1251,1377084482180669449,2021-03-31 08:10:00+05:45,https://tkpo.st/2PE3dsC
1252,1377076020616916992,2021-03-31 07:36:22+05:45,https://tkpo.st/3um2POH


In [28]:
df_['article'] = df_.urls.progress_apply(article_ktmpost)

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute

In [29]:
df_ = df_[df_['article']!=404]

In [30]:
df_

,id,created_at,urls,article
0,1393698959324028930,2021-05-16 04:30:00+05:45,http://tkpo.st/3tMY9jZ,"Valley Government preaches but does not act, s..."
1,1393668760322539521,2021-05-16 02:30:00+05:45,http://tkpo.st/2RbYAax,Climate & Environment Pandemic and restriction...
2,1393631011800449024,2021-05-16 00:00:00+05:45,http://tkpo.st/2RZnTN2,Sudurpaschim Province Flimsy huts set up in li...
3,1393615912071307267,2021-05-15 23:00:00+05:45,http://tkpo.st/3tNunvz,National Lawmakers get second dose of vaccine ...
4,1393600812832657409,2021-05-15 22:00:00+05:45,http://tkpo.st/3eSSa9j,Karnali Province Mugu ill-equipped to deal wit...
...,...,...,...,...
1249,1377099581977030659,2021-03-31 09:10:00+05:45,https://tkpo.st/31yTQgs,National Talks between the government and Thar...
1250,1377093297546657792,2021-03-31 08:45:01+05:45,https://tkpo.st/3udXpVT,National Judiciary in the spotlight as politic...
1251,1377084482180669449,2021-03-31 08:10:00+05:45,https://tkpo.st/2PE3dsC,Climate & Environment Worsening wildfires a re...
1252,1377076020616916992,2021-03-31 07:36:22+05:45,https://tkpo.st/3um2POH,Editorial Continuum of riskWe must strictly en...


In [31]:
df = df.append(df_)

In [33]:
# df['level1'] = np.nan
# df['level2'] = np.nan
# df['level3'] = np.nan

In [34]:
df['level1'] = df.apply(lambda x: (level1_count(x['article']) if pd.isnull(x.level1) else x.level1), axis=1)

In [35]:
df['level1_count'] = df.level1.apply(level_len)

In [36]:
df['level_2_3_valid'] = df['article'].apply(level_2_3_filter)

In [37]:
df['level2'] = df.apply(lambda x: (level2_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level2) else x.level2),
                                    axis=1)


In [38]:
df['level2_count'] = df.level2.apply(level_len)

In [39]:
df['level3'] = df.apply(lambda x: (level3_count(x['article'
                                    ], x['level_2_3_valid'
                                    ]) if pd.isnull(x.level3) else x.level3),
                                    axis=1)

In [40]:
df['level3_count'] = df.level3.apply(level_len)

In [41]:
df.level1_count.sum()/df.shape[0]

0.07643187593666514

In [42]:
df.level2_count.sum()/df.level_2_3_valid.sum()

0.581089258698941

In [43]:
df.level3_count.sum()/df.level_2_3_valid.sum()

0.000907715582450832

In [44]:
df.to_csv('KathmanduPost_analysis.csv', index=False)

In [46]:
list(df)

['id',
 'created_at',
 'urls',
 'article',
 'level1',
 'level2',
 'level3',
 'level1_count',
 'level_2_3_valid',
 'level2_count',
 'level3_count']

In [53]:
df[['urls',
 'created_at',
 'level1_count',
 'level_2_3_valid',
 'level2_count',
 'level3_count']].to_csv("KathmanduPost_flask.csv", index=False)